### Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -u "/content/drive/My Drive/Brain_Tumor_Classifiction.zip" -d "/content/drive/My Drive/Brain_Tumor_Classification-folder" # All data
#!unzip -u "/content/drive/My Drive/Brain_Tumor_Classification-.zip" -d "/content/drive/My Drive/Brain_Tumor_Classification--folder" # 600 in train, 280 test
#!unzip -u "/content/drive/My Drive/BreaKHis-400X.zip" -d "/content/drive/My Drive/BreaKHis400X-folder" # 600 in train, 280 test
#!unzip -u "/content/drive/My Drive/data_tumor.zip" -d "/content/drive/My Drive/dataset"
#!unzip -u "/content/drive/My Drive/Brain_Tumor_Classification---folder.zip" -d "/content/drive/My Drive/dataset_"

In [3]:
# import the necessary packages

import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,Dropout

import imutils
from tensorflow.keras.layers import BatchNormalization
import os
import sys
import seaborn as sns


In [4]:
path="/content/drive/My Drive/Brain_Tumor_Classification--folder/Brain_Tumor_Classification-"
#path="/content/drive/My Drive/Brain_Tumor_Classification--folder/Brain_Tumor_Classification-"
#path="/content/drive/My Drive/dataset_/Brain_Tumor_Classification---folder/Brain_Tumor_Classification-"
#path="/content/drive/My Drive/Brain_Tumor_Classification-folder"

path_training = path+"/Training"
path_test = path+"/Testing"
# Read input images and assign labels based on folder names
print(os.listdir(path))

SIZE = 224  #Resize images

['Testing', 'Training']


In [5]:

def croped(image_to_crop):
    # load the image, convert it to grayscale, and blur it slightly
    image = cv2.imread(image_to_crop)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    # threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    # find contours in thresholded image, then grab the largest
    # one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    # determine the most extreme points along the contour
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    # draw the outline of the object, then draw each of the
    # extreme points, where the left-most is red, right-most
    # is green, top-most is blue, and bottom-most is teal
    cv2.drawContours(image, [c], -1, (0, 255, 255), 2)
    cv2.circle(image, extLeft, 8, (0, 0, 255), -1)
    cv2.circle(image, extRight, 8, (0, 255, 0), -1)
    cv2.circle(image, extTop, 8, (255, 0, 0), -1)
    cv2.circle(image, extBot, 8, (255, 255, 0), -1)
    # show the output image
    return image

### Feature Extraction

In [6]:
def Image_to_vect(path_to_images, image_type):
  #Capture training data and labels into respective lists
  train_images = []
  train_labels = [] 
  for directory_path in glob.glob(path_to_images+"/*"):
      label = directory_path.split("/")[-1]
      print(label)
      for img_path in glob.glob(os.path.join(directory_path, "*."+image_type)):
          img = croped(img_path)
          #img = cv2.imread(img, cv2.IMREAD_COLOR)       
          img = cv2.resize(img, (SIZE, SIZE))
          
          #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
          train_images.append(img)
          train_labels.append(label)


  #Convert lists to arrays        
  train_images = np.array(train_images)
  train_labels = np.array(train_labels)
  return train_images,train_labels

In [7]:
train_images,train_labels= Image_to_vect(path_training,"jpg")
test_images,test_labels= Image_to_vect(path+"/Testing","jpg")

pituitary_tumor
no_tumor
meningioma_tumor
glioma_tumor
glioma_tumor
meningioma_tumor
no_tumor
pituitary_tumor


In [8]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

###################################################################
# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

In [9]:
from keras.applications.densenet import DenseNet121,DenseNet169,DenseNet201

def Extract_feature_DenseNet(X, i):
  #############################
  from keras.applications.densenet import DenseNet121,DenseNet169

  #Load model wothout classifier/fully connected layers
  if(i== 169):
    model = DenseNet169(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
  elif(i==201):
    model = DenseNet201(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
  
  #Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
  for layer in model.layers:
    layer.trainable = False
      
  model.summary()  #Trainable parameters will be 0


  #Now, let us use features from convolutional network for RF
  #feature_extractor=DenseNet169_model.predict(datagen.flow(x_train, y_train))
  feature_extractor=model.predict(X)


  features = feature_extractor.reshape(feature_extractor.shape[0], -1)

  X_for_training = features #This is our X input to RF

  return features

  # Train the model on training data


In [10]:
X_for_training = Extract_feature_DenseNet(x_train, 169)
#Send test data through same feature extractor process
X_test_features = Extract_feature_DenseNet(x_test,169)


Model: "densenet169"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                        

In [11]:
eval_set = [(X_for_training, train_labels_encoded),(X_test_features,test_labels_encoded)]
X_for_training.shape

(1200, 81536)

In [12]:
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
        lb = LabelBinarizer()
        lb.fit(y_test)
        y_test = lb.transform(y_test)
        y_pred = lb.transform(y_pred)
        return metrics.roc_auc_score(y_test, y_pred, average=average)
        
def evaluate_model_performance_valid_data(y_orig, y_pred):
        accuracy = metrics.accuracy_score(y_orig, y_pred)
        print('Accuracy is:',accuracy)

        # For precision of each class individually use average= None
        precision = metrics.precision_score(y_orig, y_pred, average='macro')
        print('Precision is:',precision)

        # For recall of each class individually use average= None
        recall = metrics.recall_score(y_orig, y_pred, average='macro')
        print('Recall is:',recall)

        # For recall of each class individually use average= None
        f1_score_value = metrics.f1_score(y_orig, y_pred, average='macro')
        print('F1 Score is:',f1_score_value)
        
        # AUC Scores
        auc_score = multiclass_roc_auc_score(y_orig, y_pred)
        print('AUC score is:', auc_score)

        cm = metrics.confusion_matrix(y_orig, y_pred)
        print(cm)

        plt.figure(figsize=(5,5))
        sns.heatmap(cm, annot=True, fmt=".2f", linewidths=.5, square = True, cmap = 'Blues');
        plt.ylabel('Actual label');
        plt.xlabel('Predicted label');
        all_sample_title = 'AUC Score: {:.2f}'.format(auc_score)
        plt.title(all_sample_title, size = 12)

### Classification with Xgboost

In [ ]:
import time
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import xgboost as xgb
dtrain = xgb.DMatrix(data=X_for_training, label=train_labels_encoded)
dtest = xgb.DMatrix(data=X_test_features)

# grid search
model_xgboost = XGBClassifier()
model_xgboost.fit(X_for_training,train_labels_encoded)

pred = model_xgboost.predict(X_test_features)
print(classification_report(test_labels_encoded, pred))

'''
param_grid = {'objective':  ['multi:softmax'],
        'max_depth': [3, 6, 9, 12],
        'min_child_weight': np.arange(0.0005, 0.5, 0.005),
        'gamma': np.arange(0.0,10.0,0.1),
        'learning_rate': np.arange(0.0005,0.1,0.005),
        'subsample': np.arange(0.01,1.0,0.04),
        'colsample_bylevel': np.round(np.arange(0.1,1.0,0.01)),
        'colsample_bytree': np.arange(0.1,1.0,0.01)}


kfold = KFold(n_splits=10, shuffle=True, random_state=10)
grid_search = RandomizedSearchCV(model, param_grid, scoring="accuracy", n_iter = 10, cv=kfold)
grid_result = grid_search.fit(X_for_training,train_labels_encoded)

# summarize results
print(f'Best score = {grid_result.best_score_}')
print("Best score: %f \nUsing %s \n" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_[ 'mean_test_score' ]
stds = grid_result.cv_results_[ 'std_test_score' ]
params = grid_result.cv_results_[ 'params' ]

print(f' Elapsed Time = {time.time()-start_time}')'''


In [ ]:
'''from sklearn.metrics import classification_report
print("---------------------------------------  Train -------------------------------------------------------- \n")
print(classification_report(train_labels_encoded, grid_result.best_estimator_.predict(X_for_training), target_names=le.classes_))
print("---------------------------------------  Test -------------------------------------------------------- \n")

print(classification_report(test_labels_encoded, grid_result.best_estimator_.predict(X_test_features), target_names=le.classes_))'''

### **SVM(RBF)**

In [ ]:
basic_model = SVC()
basic_model.fit(X_for_training,train_labels_encoded)

pred = basic_model.predict(X_test_features)
print(classification_report(test_labels_encoded, pred))

In [72]:
from sklearn.svm import SVC
import time
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit



start_time=time.time()

svc_model = SVC()

param_grid = {'C':np.logspace(-2, 5, 7),'gamma': np.logspace(-10, 1, 13),'kernel':['rbf']}
scorer = metrics.make_scorer(metrics.f1_score, average = 'weighted')

kfold = KFold(n_splits=5, shuffle=True, random_state=10)
model_random_search = RandomizedSearchCV(estimator=svc_model,param_distributions=param_grid, scoring='f1_micro',n_iter=5, cv=kfold)
model_random_search.fit(X_for_training,train_labels_encoded)

print(
    "The best parameters are %s with a score of %0.2f"
    % (model_random_search.best_params_, model_random_search.best_score_)
)

print(f' Elapsed Time = {time.time()-start_time}')


The best parameters are {'kernel': 'rbf', 'gamma': 1e-10, 'C': 100000.0} with a score of 0.91
 Elapsed Time = 455.10442781448364


In [73]:
model_refit = SVC(kernel= 'rbf', gamma = 1e-10, C= 100000.0)
model_refit.fit(X_for_training, train_labels_encoded)
pred = model_refit.predict(X_test_features)
print(classification_report(test_labels_encoded, pred))


              precision    recall  f1-score   support

           0       0.52      0.20      0.29        70
           1       0.51      0.57      0.54        70
           2       0.52      0.91      0.67        70
           3       0.74      0.56      0.63        70

    accuracy                           0.56       280
   macro avg       0.57      0.56      0.53       280
weighted avg       0.57      0.56      0.53       280



In [32]:
# Predict on the test set and call accuracy
y_pred_random = model_random_search.predict(X_test_features)
from sklearn.metrics import classification_report

print("---------------------------------------  Test -------------------------------------------------------- \n")

print(classification_report(test_labels_encoded, y_pred_random))


---------------------------------------  Test -------------------------------------------------------- 

              precision    recall  f1-score   support

           0       0.72      0.30      0.42        70
           1       0.55      0.67      0.61        70
           2       0.59      0.91      0.72        70
           3       0.79      0.64      0.71        70

    accuracy                           0.63       280
   macro avg       0.66      0.63      0.61       280
weighted avg       0.66      0.63      0.61       280

